# Navigation
---
This notebook is based on the notebook of DQN coding exercice of udacity nanodegree deep reinforcement learning.

We will train an agent to navigate in banana environment.

### 1. Import the Necessary Packages

In [ ]:
from unityagents import UnityEnvironment
from collections import deque
from agents import AgentQ
from models import QNetwork
import torch, os, numpy as np, torch.optim as optim, matplotlib.pyplot as plt
%matplotlib inline

### 2. Instantiate the Environment and Agent

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

Initialize the environment in the code cell below.

In [ ]:
env = UnityEnvironment(file_name="Banana_Windows_x86_64/Banana.exe")

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of actions
action_size = brain.vector_action_space_size

# the state size
state_size = len(env_info.vector_observations[0])

In [ ]:
seed = 0
# initialize networks
qnetwork_local = QNetwork(state_size, action_size, seed)
qnetwork_target = QNetwork(state_size, action_size, seed)

optimizer = optim.Adam  # get an opimizer
lr = 5e-4               # learning rate 
param_opt = dict(lr=lr) # the optimizer parameters
agent = AgentQ(qnetwork_local, qnetwork_target, optimizer, param_opt, action_size, seed,
               device="cuda", dqn="double", a=0.0, b=0.0) # define the agent
check = "Navigation_checkpoint" # the checkpoint folder

### 3. Train the Agent


In [ ]:
def trainer(agent,path,n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        agent: the agent to train
        path: checkpoint path
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        env_info = env.reset(train_mode=True)[brain_name] # reset the environment
        state = env_info.vector_observations[0]            # get the current state
        score = 0
        for t in range(max_t):
            action= agent.act(state, eps)
            env_info = env.step(action)[brain_name]        # send the action to the environment
            next_state = env_info.vector_observations[0]   # get the next state
            reward = env_info.rewards[0]                   # get the reward
            done = env_info.local_done[0]                  # see if episode has finished
            agent.step(state, action, reward, next_state, done) 
            state = next_state
            score += reward
            if done:
                break
                
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        mean=np.mean(scores_window)
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, mean), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, mean))
            save_path=os.path.join(path,'checkpoint_'+str(mean)+'.pth')
            torch.save(agent.qnetwork_local.state_dict(),save_path)
            
    return scores



In [ ]:
scores = trainer(agent,check)

In [ ]:
# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

### 4. Watch the Smart Agent!

In the next code cell, we will load the trained weights and evaluate the model.

In [ ]:
# load the weights from file

## get all checkpoints
checkpoints=[f for f in os.listdir(check) if f.startswith('checkpoint_') and f.endswith('.pth') ]

## find the index of maximum score
index=np.argmax([float(f[11:-4]) for f in checkpoints])

## take the optimal checkpoint
name=checkpoints[index]

print("Weights loaded from",name)

path=os.path.join(check,name)

agent.qnetwork_local.load_state_dict(torch.load(path))

for i in range(10):
    env_info = env.reset(train_mode=False)[brain_name]      # reset the environment
    state = env_info.vector_observations[0]                 # get the current state
    score=0                                                 # initialise score
    for j in range(1000):
        action = agent.act(state)
        env_info = env.step(action)[brain_name]        # send the action to the environment
        state = env_info.vector_observations[0]        # get the next state
        score += env_info.rewards[0]                   # update score
        done = env_info.local_done[0]                  # see if episode has finished
        if done:
            print("trail :",i+1,"score :",score)
            break 
            
env.close()